In [1]:
!git clone https://github.com/KhoiDang-KHDL/DS201_Lab04

fatal: destination path 'DS201_Lab04' already exists and is not an empty directory.


In [2]:
import sys
sys.path.append('/kaggle/working/DS201_Lab04') 

# Check thư mục
import os
print(os.listdir('/kaggle/working/DS201_Lab04'))

['config', '.git', 'checkpoints', 'dataset_construct', 'test.py', 'train.py', 'dataset', 'model']


In [3]:
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!pip install evaluate pyyaml tqdm
!pip install protobuf==3.20.3

In [5]:
%cd DS201_Lab04
base_path = os.getcwd()
base_path

/kaggle/working/DS201_Lab04


'/kaggle/working/DS201_Lab04'

In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.auto import tqdm 

import evaluate


SETTINGS = {
    "data_dir": "dataset",  
    "src_lang": "english",
    "tgt_lang": "vietnamese",
    "batch_size": 32,       
    "lr": 0.0001,
    "epochs": 30,           
    "patience": 3,          
    "hidden_dim": 256,
    "num_layers": 3,
    "dropout": 0.1,
    "weight_decay": 0.00001,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

print(f"Running on device: {SETTINGS['device']}")


2025-12-17 05:19:49.759953: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765948789.781058     147 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765948789.787539     147 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Running on device: cuda


In [7]:
from dataset_construct.vocab import Vocabulary
from dataset_construct.PhoMT_data import TranslationDataset, translation_collate_fn

from model.bai1 import Seq2Seq
from model.bai2 import Seq2SeqBahdanau
from model.bai3 import Seq2SeqLuong

class DirectTrainer:
    def __init__(self, model_name):
        self.model_name = model_name
        self.device = SETTINGS["device"]
        self.metric = evaluate.load("rouge")
        
        # 1. Prepare Data
        self._prepare_data()
        
        # 2. Build Model
        self._build_model()
        
        # 3. Optimizer & Loss
        self.optimizer = optim.Adam(self.model.parameters(), lr=SETTINGS["lr"])
        self.criterion = nn.CrossEntropyLoss(ignore_index=self.vocab.pad_idx)
        
    def _prepare_data(self):
        print(f"[{self.model_name.upper()}] Loading Data...")
        data_dir = SETTINGS["data_dir"]
        
        # Load Vocab
        self.vocab = Vocabulary(
            f"{data_dir}/small-train.json", 
            SETTINGS["src_lang"], 
            SETTINGS["tgt_lang"]
        )
        
        # Load Datasets
        train_ds = TranslationDataset(f"{data_dir}/small-train.json", self.vocab)
        dev_ds   = TranslationDataset(f"{data_dir}/small-dev.json", self.vocab)
        test_ds  = TranslationDataset(f"{data_dir}/small-test.json", self.vocab)
        
        # Dataloaders
        self.train_loader = DataLoader(train_ds, batch_size=SETTINGS["batch_size"], shuffle=True, collate_fn=translation_collate_fn)
        self.dev_loader   = DataLoader(dev_ds, batch_size=SETTINGS["batch_size"], shuffle=False, collate_fn=translation_collate_fn)
        self.test_loader  = DataLoader(test_ds, batch_size=SETTINGS["batch_size"], shuffle=False, collate_fn=translation_collate_fn)
        
    def _build_model(self):
        print(f"[{self.model_name.upper()}] Building Model...")
        if self.model_name == "bai1":
            self.model = Seq2Seq(self.vocab, SETTINGS["hidden_dim"], SETTINGS["num_layers"], SETTINGS["dropout"])
        elif self.model_name == "bai2":
            self.model = Seq2SeqBahdanau(self.vocab, SETTINGS["hidden_dim"], SETTINGS["num_layers"], SETTINGS["dropout"])
        elif self.model_name == "bai3":
            self.model = Seq2SeqLuong(self.vocab, SETTINGS["hidden_dim"], SETTINGS["num_layers"], SETTINGS["dropout"])
        else:
            raise ValueError("Model name must be 'bai1', 'bai2', or 'bai3'")
            
        self.model.to(self.device)

    def train_one_epoch(self):
        self.model.train()
        total_loss = 0
        
        for batch in tqdm(self.train_loader, desc="Training", leave=False):
            src = batch["src"].to(self.device)
            tgt = batch["tgt"].to(self.device)
            src_len = batch["src_len"].to(self.device)
            
            self.optimizer.zero_grad()
            output = self.model(src, src_len, tgt, teacher_forcing=True)
            
            # Flatten
            output = output[:, 1:].reshape(-1, output.shape[-1])
            tgt = tgt[:, 1:].reshape(-1)
            
            loss = self.criterion(output, tgt)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()
            
            total_loss += loss.item()
            
        return total_loss / len(self.train_loader)

    @torch.no_grad()
    def evaluate(self, loader):
        self.model.eval()
        total_loss = 0
        preds, refs = [], []
        
        for batch in tqdm(loader, desc="Evaluating", leave=False):
            src = batch["src"].to(self.device)
            tgt = batch["tgt"].to(self.device)
            src_len = batch["src_len"].to(self.device)
            
            # Calc Loss
            out = self.model(src, src_len, tgt, teacher_forcing=True)
            loss = self.criterion(out[:, 1:].reshape(-1, out.shape[-1]), tgt[:, 1:].reshape(-1))
            total_loss += loss.item()
            
            # Inference Text
            pred_ids = self.model.inference(src, src_len)
            for i in range(src.size(0)):
                preds.append(self.vocab.indices_to_text(pred_ids[i], self.vocab.tgt_lang))
                refs.append(self.vocab.indices_to_text(tgt[i], self.vocab.tgt_lang))
                
        rouge = self.metric.compute(predictions=preds, references=refs)
        return total_loss / len(loader), rouge["rougeL"], preds, refs

    def run(self):
        best_score = 0 # Hoặc set -1 để chắc chắn lưu ít nhất 1 lần
        no_improve = 0
        save_path = f"checkpoints/{self.model_name}.pt"
        os.makedirs("checkpoints", exist_ok=True)
        
        print(f"\n START TRAINING {self.model_name.upper()}")
        for epoch in range(1, SETTINGS["epochs"] + 1):
            train_loss = self.train_one_epoch()
            val_loss, val_rouge, _, _ = self.evaluate(self.dev_loader)
            
            print(f"Epoch {epoch}: Train Loss={train_loss:.4f} | Val Loss={val_loss:.4f} | ROUGE-L={val_rouge:.4f}")
            
            # Sửa logic lưu model: Lưu nếu tốt hơn HOẶC là epoch đầu tiên (để tránh lỗi không có file)
            if val_rouge > best_score:
                best_score = val_rouge
                no_improve = 0
                torch.save(self.model.state_dict(), save_path)
                print("  >>> Saved Best Model!")
            else:
                no_improve += 1
                if no_improve >= SETTINGS["patience"]:
                    print(" Early Stopping ")
                    break
        
        # Load best model to test (CHỈ LOAD NẾU FILE TỒN TẠI)
        print(f"\n TESTING {self.model_name.upper()} <<<")
        if os.path.exists(save_path):
            self.model.load_state_dict(torch.load(save_path))
            print("  >>> Loaded Best Checkpoint.")
        else:
            print("  >>> Warning: No checkpoint found (Model didn't learn well). Using last epoch state.")

        test_loss, test_rouge, preds, refs = self.evaluate(self.test_loader)
        
        print(f"FINAL TEST ROUGE-L: {test_rouge:.4f}")
        print("\n Example Predictions")
        for i in range(min(3, len(preds))):
            print(f"Ref : {refs[i]}")
            print(f"Pred: {preds[i]}")
            print("-" * 20)

In [8]:
def main():
    # #  Bài 1
    print("\n" + "="*40)
    trainer1 = DirectTrainer("bai1")
    trainer1.run()
    
    # #  Bài 2
    print("\n" + "="*40)
    trainer2 = DirectTrainer("bai2")
    trainer2.run()
    
    #  Bài 3
    print("\n" + "="*40)
    trainer3 = DirectTrainer("bai3")
    trainer3.run()

if __name__ == "__main__":
    main()


[BAI1] Loading Data...
[BAI1] Building Model...

 START TRAINING BAI1


Training:   0%|          | 0/625 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 1: Train Loss=6.4039 | Val Loss=6.3559 | ROUGE-L=0.1707
  >>> Saved Best Model!


Training:   0%|          | 0/625 [00:00<?, ?it/s]

...

Training:   0%|          | 0/625 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 8: Train Loss=5.4241 | Val Loss=5.7696 | ROUGE-L=0.1871
 Early Stopping 

 TESTING BAI3 <<<
  >>> Loaded Best Checkpoint.


Evaluating:   0%|          | 0/63 [00:00<?, ?it/s]

FINAL TEST ROUGE-L: 0.2443

 Example Predictions
Ref : anh albert và chị susan , thuộc hội thánh west ở tuscaloosa , alabama
Pred: và là , có có có có có có có , , một , , một , những những những những những một những một .
--------------------
Ref : ngày 11 và 12 - 1 - 2020 , những cơn bão lớn đã quét qua và phá huỷ nhiều vùng ở miền nam và miền trung hoa kỳ .
Pred: và tôi , có có có có , , , , , , một , , những những những những những những .
--------------------
Ref : những trận mưa to và gió lớn trong suốt hai ngày cùng với nhiều cơn lốc xoáy đã gây thiệt hại nặng nề cho nhiều bang .
Pred: và là , , , , , , , một , , một , , những những những những những những một những một một .
--------------------
